# Creating DynamoDB Resources and skeleton for bandits

## Resources

- https://hands-on.cloud/working-with-dynamodb-in-python-using-boto3
- https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.html
- https://github.com/aws-samples/chalice-workshop (see example code for todo-app) & [relevant workshop section](https://chalice-workshop.readthedocs.io/en/latest/todo-app/part1/03-todo-app-dynamodb.html) for better integration with Chalice


In [ ]:
from pprint import pprint

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"]
os.environ["AWS_DEFAULT_REGION"]

'us-east-2'

In [ ]:
import boto3
ddb = boto3.resource('dynamodb')

## Managing Tables

Only define index/key attributes when creating tables

In [ ]:
BANDIT_DATA_TABLE = 'tg-bandits-bandit1'

In [ ]:
# table = ddb.create_table (
#     TableName = BANDIT_DATA_TABLE,
#        KeySchema = [
#            {
#                'AttributeName': 'bandit_id',
#                'KeyType': 'HASH'
#            },
#            {
#                'AttributeName': 'creation_date',
#                'KeyType': 'RANGE'
#            }
#            ],
#            AttributeDefinitions = [
#                {
#                    'AttributeName': 'bandit_id',
#                    'AttributeType': 'S'
#                },
#                {
#                    'AttributeName':'creation_date',
#                    'AttributeType': 'S'
#                }
#             ],
#             ProvisionedThroughput={
#                 'ReadCapacityUnits':1,
#                 'WriteCapacityUnits':1
#             }
          
#     )


Clearing table (taking into account that there might be many items). Note both keys

In [ ]:
table = ddb.Table(BANDIT_DATA_TABLE)
table_scan = table.scan()
with table.batch_writer() as cursor:
    for item in table_scan['Items']:
        cursor.delete_item(
            Key={
                'bandit_id': item['bandit_id'],
                'creation_date': item['creation_date']
            }
        )

## Manipulating Items

See [docs](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.03.html) for details

Put initial bandit data into table. `boto3` does not process float data, so we will use Decimal + json trick from [this advice on Ruan Bekker's blog](https://blog.ruanbekker.com/blog/2019/02/05/convert-float-to-decimal-data-types-for-boto3-dynamodb-using-python/)

In [ ]:
from decimal import Decimal
import json
bandit_key = {'bandit_id': 'test_epsilon_greedy_1',
                                'creation_date': '2021-01-07 00:00:01'}
bandit_def = { 
     'bandit_id': 'test_epsilon_greedy_1',
     'creation_date': '2021-01-07 00:00:01',
     'type': 'egreedy',
     'state': {'arms': {'arm1': {'text': 'this is arm _1_', 'reward': 0.0, 'n':0}, 'arm2': {'text': 'this is arm *2*', 'reward': 0.0, 'n':0},}},
     'epsilon': 0.5
      }
bandit_def.update(bandit_key)

badnit_def_dec = json.loads(json.dumps(bandit_def), parse_float=Decimal)

pprint(badnit_def_dec)

{'bandit_id': 'test_epsilon_greedy_1',
 'creation_date': '2021-01-07 00:00:01',
 'epsilon': Decimal('0.5'),
 'state': {'arms': {'arm1': {'n': 0,
                             'reward': Decimal('0.0'),
                             'text': 'this is arm _1_'},
                    'arm2': {'n': 0,
                             'reward': Decimal('0.0'),
                             'text': 'this is arm *2*'}}},
 'type': 'egreedy'}


In [ ]:
response = table.put_item(
Item = badnit_def_dec
)
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 08 Jan 2022 04:35:02 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2745614147',
                                      'x-amzn-requestid': '82U63GBHLLBR7P34AB28J8SOQRVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '82U63GBHLLBR7P34AB28J8SOQRVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [ ]:
response = table.get_item(Key=bandit_key)
# pprint(response)
item = response['Item']
pprint(item)

{'Item': {'bandit_id': 'test_epsilon_greedy_1',
          'creation_date': '2021-01-07 00:00:01',
          'epsilon': Decimal('0.5'),
          'state': {'arms': {'arm1': {'n': Decimal('0'),
                                      'reward': Decimal('0'),
                                      'text': 'this is arm _1_'},
                             'arm2': {'n': Decimal('0'),
                                      'reward': Decimal('0'),
                                      'text': 'this is arm *2*'}}},
          'type': 'egreedy'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '330',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 08 Jan 2022 04:35:07 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '537686459',
                                      'x-amzn-r

Incrementing `reward` and `n`

In [ ]:
item['state']['arms']['arm1']['reward'] += 1
item['state']['arms']['arm1']['n'] += 1 

{'arm1': {'n': '6', 'reward': '6', 'text': 'this is arm _1_'},
 'arm2': {'n': '0', 'reward': '0', 'text': 'this is arm *2*'}}


Replacing the item (checking key attibutes equality)

In [ ]:
response = table.put_item(Item = item)
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 08 Jan 2022 04:44:22 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2745614147',
                                      'x-amzn-requestid': 'CHH2QMKEAVBQJ43TKU9532TCJVVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'CHH2QMKEAVBQJ43TKU9532TCJVVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


Draft of more incremental item update below

In [ ]:
# # https://stackoverflow.com/a/68305349/119759
# class DecimalEncoder(json.JSONEncoder):
#   def default(self, obj):
#     if isinstance(obj, Decimal):
#       return str(obj)
#     return json.JSONEncoder.default(self, obj)

# arms_updated = json.loads(json.dumps(item['state']['arms'], cls=DecimalEncoder), parse_float=Decimal)

# pprint(arms_updated)
# response = table.update_item(
#     Key= bandit_key,
#     UpdateExpression="set state.arms = :arms",
#     ExpressionAttributeValues={
#         ':arms': arms_updated
#     },
#     ReturnValues="UPDATED_NEW"
# )


# # response = table.update_item(
# #     Key={
# #         'year': year,
# #         'title': title
# #     },
# #     UpdateExpression="set info.rating = info.rating + :val",
# #     ExpressionAttributeValues={
# #         ':val': Decimal(rating_increase)
# #     },
# #     ReturnValues="NONE
# # )
# pprint(response)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bf5284cd-2333-4dd7-a06b-b1d0e03db006' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>